# staff.py

Purpose: output SQL statement that can be used to get staff members that fit the characteristics given

- Build Python code staff.py to extract staffing using same configuration file as pull.py
- Usage: staff.py config.cfg
- Outputs staffing levels to STDOUT

- Q: what should this look like? For now:

- For each day output
    - For each hcw/uid output 3 types of records
        - hcw, 'day', nday, uid, #days-worked, jtid, #visits, #unique-rooms, min(itime), max(otime), max(otime)-min(itime) [#days-worked always 1]
        - hcw, 'week', nweek, uid, #days-worked, jtid, #visits, #unique-rooms, min(itime), max(otime), max(otime)-min(itime)   [only complete weeks]
        - hcw, 'all', ndays, uid, #days-worked, jtid, #visits, #unique-rooms, min(itime), max(otime), max(otime)-min(itime)    [over the entire interval]

- where nday is an integer 0-6 and Mon=0, Tue=1... 
    - and #days-worked always 1 (only one day for this kind of record) 
    - and nweek is an integer (the "iso week" of the year) 
    - and ndays is an integer counting number of days in the interval

- so the output will be a bunch of csv-like lines. 
- The second field determines the line type, and the third field has a type-dependent interpretation. 
- So if a HCW works 3 days/week for the 2.5 weeks weeks in the pull interval, we should see:
    - 3*2.5 = ~8 lines of the first type [e.g., 'Mon' or 'Sat']
    - 2 lines of the second type ['week'], each Mon-Sun
    - 1 line of the third type ['all']

Generates a mySQL query for the ssense Database that returns information from the Database in regards to hcws.
The specific attributes and values within are extracted via text file, who's name or path is the only parameter.
The file should be formatted where each line denotes a specification of what attribute you want and what value 
you want from the attribute. 

If there are no such lines, the query generated would be a viewing of ssense.visits.
It is assumed when referring to dates that if the year is last, then month preceeds day.

In [ ]:
# CONFIGURATION FILE CSV INPUTS
# hid
# start
# end

# TABLE RETURNS:
# hid
# uid
# jtid
# num_unique_rooms
# num_visits

# min_itime
# max_otime
# max_otime_min_itime

# begin_date
# end_date
# nday
# nweek
# ndays

In [ ]:
# import mysql.connector as cnxn
# connection = cnxn.connect(host = "vinci.cs.uiowa.edu", database = "ssense", user = "madjin", password = "madjin")

In [2]:
# imports
from dateutil import parser
from datetime import date
from pandas import to_datetime
from datetime import datetime
import datetime
import re
import argparse
from dateutil.parser import parse

In [48]:
filename = 'staff_tester.txt'
use_start = False
use_end = False

with open(filename, 'r') as config:
    select_values = 'v.hid, "day" AS period, u.uid, j.jtid, DATE(v.itime) AS date, DAYOFWEEK(v.itime) AS nday, WEEKOFYEAR(v.itime) AS week, COUNT(DISTINCT v.rid) AS num_rooms, COUNT(DISTINCT v.vid) AS num_visits, MIN(v.itime), MAX(v.otime), 1 AS ndays'
    # DAYOFWEEK(date) AS ndays, \
    from_values = 'visits v, hcws h, rooms r, units u, jobs j'
    where_values = 'v.hid=h.hid AND v.rid=r.rid AND r.uid=u.uid AND h.jid=j.jid AND'
    group_by_values = 'v.hid, date'
    
    for line in config:
        line = [part.strip().lower() for part in line.split(':')]

        match line[0]:
            case 'hid':
                hid = line[1]
                try:
                    hid_list = []
                    for a, b in re.findall(r'(\d+)-?(\d*)', str(hid)):
                        hid_list.extend(range(int(a), int(a)+1 if b=='' else int(b)+1))
                    where_values += " v.hid in ("
                    for item in hid_list:
                        where_values += str(item) + ","
                    where_values = " " + where_values[:-1] + ") AND "
                except:
                    print('invalid format: {} is an invalid specification for attribute {}'.format(info[1:], info[0]))
                    break
                
            case 'start':
                start = parser.parse(':'.join(line[1:]))
                use_start = True

            case 'end':
                end = parser.parse(':'.join(line[1:]))
                use_end = True
            
            case '#':
                pass

# start and end values
if use_start==True and use_end==True:
    select_values += f', TIMEDIFF(MAX(v.otime), MIN(v.itime)) as time_diff'
    where_values += f'v.itime >= "{start}" AND v.otime <= "{end}"'.format(start, end)
    ndays = (end-start).days
elif use_start:
    select_values += f', MAX(v.otime)-MIN(v.itime) as time_diff'
    where_values += f'v.itime >= "{start}" AND '
    ndays = (end-start).days
elif use_end:
    select_values += f', MAX(v.otime)-MIN(v.itime) as time_diff'
    where_values += f'v.otime <= "{end}" AND '
    ndays = (end-start).days
                      
# Putting the pieces together
select_values = select_values.strip(' AND')
from_values = from_values.strip(' AND')
where_values = where_values.strip(' AND')
group_by_values = group_by_values.strip(' AND')
sql_statement = 'SELECT '+select_values+'\n'+'FROM '+from_values+'\n'+'WHERE '+where_values+'\n'+'GROUP BY '+group_by_values + ';'
#sql_statement = 'SELECT '+select_values+' '+'FROM '+from_values+' '+'WHERE '+where_values+' '+'GROUP BY '+group_by_values + ';'

print(sql_statement)

SELECT v.hid, "day" AS period, u.uid, j.jtid, DATE(v.itime) AS date, DAYOFWEEK(v.itime) AS nday, WEEKOFYEAR(v.itime) AS week, COUNT(DISTINCT v.rid) AS num_rooms, COUNT(DISTINCT v.vid) AS num_visits, MIN(v.itime), MAX(v.otime), 1 AS ndays, TIMEDIFF(MAX(v.otime), MIN(v.itime)) as time_diff
FROM visits v, hcws h, rooms r, units u, jobs j
WHERE v.hid=h.hid AND v.rid=r.rid AND r.uid=u.uid AND h.jid=j.jid AND v.hid in (10,11,12) AND v.itime >= "2017-01-01 00:00:00" AND v.otime <= "2018-12-01 00:00:00"
GROUP BY v.hid, date;
